In [1]:
from hypersurface_tf import *
from generate_h import *
from biholoNN import *
import tensorflow as tf
import numpy as np

### Prepare the dataset:

In [13]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
np.random.seed(123)
HS = Hypersurface(Z, f, 100000)
np.random.seed(124)
HS_test = Hypersurface(Z, f, 10000)

In [21]:
train_set = generate_dataset(HS)
test_set = generate_dataset(HS_test)

In [22]:
train_set = train_set.shuffle(500000).batch(1000)
test_set = test_set.shuffle(500000).batch(1000)

### Build the model:

In [23]:
class KahlerPotential(tf.keras.Model):

    def __init__(self):
        super(KahlerPotential, self).__init__()
        self.biholomorphic = Biholomorphic()
        #self.layer1 = WidthOneDense()
        #self.layer1 = Dense(25, 1, activation=None)
        self.layer1 = Dense(25,500, activation=tf.square)
        self.layer2 = Dense(500,500, activation=tf.square)
        self.layer3 = Dense(500,1000, activation=tf.square)
        self.layer4 = Dense(1000,4000, activation=tf.square)
        #self.layer5 = Dense(500,500, activation=tf.square)
        #self.layer6 = Dense(500,500, activation=tf.square)

        
    def call(self, inputs):
        x = self.biholomorphic(inputs)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        #x = self.layer5(x)
        #x = self.layer6(x)
        x = tf.reduce_sum(x, 1)
        x = tf.math.log(x)
        return x

In [24]:
model = KahlerPotential()

In [25]:
@tf.function
def volume_form(x, Omega_Omegabar, mass, restriction):

    kahler_metric = complex_hessian(tf.math.real(model(x)), x)
    volume_form = tf.math.real(tf.linalg.det(tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True))))
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * volume_form / Omega_Omegabar)
    #factor = tf.constant(35.1774, dtype=tf.complex64)
    return volume_form / factor

In [ ]:
optimizer = tf.keras.optimizers.Adam()
epochs = 6000

for epoch in range(epochs):
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(train_set):
        with tf.GradientTape() as tape:
            
            omega = volume_form(points, Omega_Omegabar, mass, restriction)
            loss = weighted_MAPE(Omega_Omegabar, omega, mass)
            grads = tape.gradient(loss, model.trainable_weights)
            #print(grads)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 500 == 0:
            print("step %d: loss = %.4f" % (step, loss))
    
    test_loss = 0
    test_loss_old = 100
    total_mass = 0
    
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(test_set):
        omega = volume_form(points, Omega_Omegabar, mass, restriction)
        mass_sum = tf.reduce_sum(mass)
        test_loss += weighted_MAPE(Omega_Omegabar, omega, mass) * mass_sum
        total_mass += mass_sum
   
    test_loss = test_loss / total_mass
    print("test_loss:", test_loss.numpy())
    
    # This part doesn't work right now
    #if test_loss > test_loss_old:
    #    break
    #test_loss_old = test_loss

step 0: loss = 0.3698
test_loss: 0.013703559
step 0: loss = 0.0133
test_loss: 0.01319101
step 0: loss = 0.0136
test_loss: 0.01134931
step 0: loss = 0.0110
test_loss: 0.010593451
step 0: loss = 0.0100
test_loss: 0.011462548
step 0: loss = 0.0116
test_loss: 0.00845716
step 0: loss = 0.0089
test_loss: 0.0063734464
step 0: loss = 0.0065
test_loss: 0.0066126166
step 0: loss = 0.0065
test_loss: 0.0064976257
step 0: loss = 0.0063
test_loss: 0.00584696
step 0: loss = 0.0057
test_loss: 0.004994429
step 0: loss = 0.0052
test_loss: 0.0054956754
step 0: loss = 0.0059
test_loss: 0.0058985543
step 0: loss = 0.0060
test_loss: 0.0045970283
step 0: loss = 0.0047
test_loss: 0.0044949856
step 0: loss = 0.0043
test_loss: 0.004569483
step 0: loss = 0.0045
test_loss: 0.0046341163
step 0: loss = 0.0047
test_loss: 0.004763415
step 0: loss = 0.0048
test_loss: 0.004136487
step 0: loss = 0.0042
test_loss: 0.0039942786
step 0: loss = 0.0040
test_loss: 0.004185248
step 0: loss = 0.0045
test_loss: 0.0054799183
step